In [34]:
# I GET ERRORS OF MORRPH WITH ANY BUT THE SIMPLEST
# i have started from https://realpython.com/natural-language-processing-spacy-python/

In [35]:
import spacy
#nlp now refers to the language model instace
# it doesnt give me the morphology errors if I add the disable here
#nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser"])
nlp = spacy.load("en_core_web_sm")

In [36]:
about_text= "we went to the bak for some money"

In [37]:
#This is how you can convert a text file into a processed Doc object.
about_doc = nlp(about_text)

In [38]:
# Extract tokens for the given doc
print ([token.text for token in about_doc])
# assume that:Variable names ending with the suffix _text are Unicode string objects.
# Variable name ending with the suffix _doc are spaCy’s language model objects

['we', 'went', 'to', 'the', 'bak', 'for', 'some', 'money']


In [39]:

# In spaCy, the sents property is used to extract sentences. 
# Here’s how you would extract the total number of sentences 
# and the sentences for a given input text:
sentences = list(about_doc.sents)
print (len(sentences))
for sentence in sentences:
    print (sentence)
    print ("then ")

1
we went to the bak for some money
then 


In [40]:
#Here’s an example, where an ellipsis(...) is used as the delimiter. 
#These sentences are still obtained via the sents attribute, a
def set_custom_boundaries(doc):
     # Adds support to use `...` as the delimiter for sentence detection
     for token in doc[:-1]:
         if token.text == '...':
             doc[token.i+1].is_sent_start = True
     return doc

ellipsis_text = ('Gus, can you, ... never mind, I forgot'
                  ' what I was saying. So, do you think'
                  ' we should ...')
# Load a new model instance
custom_nlp = spacy.load('en_core_web_sm')
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)
# Sentence Detection with no customization
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
 print(sentence)


Gus, can you, ...
never mind, I forgot what I was saying.
So, do you think we should ...
Gus, can you, ... never mind, I forgot what I was saying.
So, do you think we should ...


In [41]:
#Tokenization is the next step after sentence detection. 
#In spaCy, you can print tokens by iterating on the Doc object:
for token in about_doc:
    print (token, token.idx) 
#Note how spaCy preserves the starting index of the tokens. 
#It’s useful for in-place word replacement. 
#

we 0
went 3
to 8
the 11
bak 15
for 19
some 23
money 28


In [42]:
#spaCy provides various attributes for the Token class:
for token in about_doc:
     print (token, token.idx, token.text_with_ws,
            token.is_alpha, token.is_punct, token.is_space,
            token.shape_, token.is_stop)
#In this example, some of the commonly required attributes are accessed:

#text_with_ws prints token text with trailing space (if present).
#is_alpha detects if the token consists of alphabetic characters or not.
#is_punct detects if the token is a punctuation symbol or not.
#is_space detects if the token is a space or not.
#shape_ prints out the shape of the word.
#is_stop detects if the token is a stop word or not.

we 0 we  True False False xx True
went 3 went  True False False xxxx False
to 8 to  True False False xx True
the 11 the  True False False xxx True
bak 15 bak  True False False xxx False
for 19 for  True False False xxx True
some 23 some  True False False xxxx True
money 28 money True False False xxxx False


In [43]:
#I then jog on past You can also customize the tokenization process to detect tokens on custom characters
# and stop words
# tutorial is here btw https://realpython.com/natural-language-processing-spacy-python/
#and lemmatization
# in effect .....
# Remove stop words and punctuation symbols
words = [token.text for token in about_doc
          if not token.is_stop and not token.is_punct]
from collections import Counter
word_freq = Counter(words)
# 5 commonly occurring words with their frequencies
common_words = word_freq.most_common(5)
print (common_words)
# Unique words
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)
# then i jump on pass POS tagging


[('went', 1), ('bak', 1), ('money', 1)]
['went', 'bak', 'money']


Rule-based matching is one of the steps in extracting information from unstructured text. It’s used to identify and extract tokens and phrases according to patterns (such as lowercase) and grammatical features (such as part of speech).

Rule-based matching can use regular expressions to extract entities (such as phone numbers) from an unstructured text. It’s different from extracting text using regular expressions only in the sense that regular expressions don’t consider the lexical and grammatical attributes of the text.

With rule-based matching, you can extract a first name and a last name, which are always proper nouns:
There is also an example of extracting phone numbers which I avoided

In [44]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
def extract_full_name(nlp_doc):
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('FULL_NAME', None, pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text

extract_full_name(about_doc)


ou can create a preprocessing function that takes text as input and applies the following operations:

Lowercases the text
Lemmatizes each token
Removes punctuation symbols
Removes stop words
A preprocessing function converts text to an analyzable format. It’s necessary for most NLP tasks. Here’s an example:

In [45]:
>>> def is_token_allowed(token):
...     '''
...         Only allow valid tokens which are not stop words
...         and punctuation symbols.
...     '''
...     if (not token or not token.string.strip() or
...         token.is_stop or token.is_punct):
...         return False
...     return True
...
>>> def preprocess_token(token):
...     # Reduce token to its lowercase lemma form
...     return token.lemma_.strip().lower()
...
>>> complete_filtered_tokens = [preprocess_token(token)
...     for token in about_doc if is_token_allowed(token)]
>>> complete_filtered_tokens
#Note that the complete_filtered_tokens does not contain any stop word or punctuation symbols
#symbols and consists of lemmatized lowercase tokens.

['went', 'bak', 'money']

There is then a section on dependency parsing , and navigating the consqeuent tree for it. I avoid the detail of this but then show a dependency print out further down......

Shallow Parsing
Shallow parsing, or chunking, is the process of extracting phrases from unstructured text. Chunking groups adjacent tokens into phrases on the basis of their POS tags. There are some standard well-known chunks such as noun phrases, verb phrases, and prepositional phrases.

Noun Phrase Detection
A noun phrase is a phrase that has a noun as its head. It could also include other kinds of words, such as adjectives, ordinals, determiners. Noun phrases are useful for explaining the context of the sentence. They help you infer what is being talked about in the sentence.

spaCy has the property noun_chunks on Doc object. You can use it to extract noun phrases:
I avoided the next bit about verb phrases

In [46]:

>>> # Extract Noun Phrases
>>> for chunk in about_doc.noun_chunks:
...     print (chunk)

we
the bak
some money


Named Entity Recognition (NER) is the process of locating named entities in unstructured text and then classifying them into pre-defined categories, such as person names, organizations, locations, monetary values, percentages, time expressions, and so on.

You can use NER to know more about the meaning of your text. For example, you could use it to populate tags for a set of documents in order to improve the keyword search. You could also use it to categorize customer support tickets into relevant categories.

spaCy has the property ents on Doc objects. You can use it to extract named entities:

In [47]:
for ent in about_doc.ents:
    print(ent.text, ent.start_char, ent.end_char,ent.label_, spacy.explain(ent.label_))

In the above example, ent is a Span object with various attributes:

text gives the Unicode text representation of the entity.
start_char denotes the character offset for the start of the entity.
end_char denotes the character offset for the end of the entity.
label_ gives the label of the entity.
spacy.explain gives descriptive details about an entity label. The spaCy model has a pre-trained list of entity classes. 


spaCy comes with a built-in visualizer called displaCy. You can use it to visualize a dependency parse or named entities in a browser or a Jupyter notebook.You can use displaCy to visualize these entities:

In [48]:
#Note: Here’s how you can use displaCy in a Jupyter notebook:
# first one gives dependencies
# second one gives entitites!
# it disconnects for large documents, so that is why I have added the indexes
from spacy import displacy
displacy.render(about_doc[105000:105020], style='dep', jupyter=True)
displacy.render(about_doc[104000:104020], style='ent', jupyter=True)

ZeroDivisionError: division by zero